<a href="https://colab.research.google.com/github/bhavya-kashyap/web-scrapping/blob/main/web_scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt update
!apt install chromium-chromedriver
!pip install selenium
!pip install bs4

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

In [4]:
import json
import numpy as np
import pandas as pd         
import ctypes

In [8]:
hospitals = pd.read_csv('https://raw.githubusercontent.com/bhavya-kashyap/web-scrapping/main/GA_Hospitals.csv')
#hospitals

In [9]:
def driversetup():
    options = webdriver.ChromeOptions()
    #run Selenium in headless mode
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    #overcome limited resource problems
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("lang=en")
    #open Browser in maximized mode
    options.add_argument("start-maximized")
    #disable infobars
    options.add_argument("disable-infobars")
    #disable extension
    options.add_argument("--disable-extensions")
    options.add_argument("--incognito")
    options.add_argument("--disable-blink-features=AutomationControlled")
    
    driver = webdriver.Chrome(options=options)

    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined});")

    return driver

def pagesource(url, driver):
    driver = driver
    driver.get(url)
    time.sleep(10)
    soup = BeautifulSoup(driver.page_source)
    #driver.close()
    return soup


In [10]:
d = driversetup()

In [15]:
url = 'https://freida.ama-assn.org/institution/120173'
url1 = 'https://freida.ama-assn.org/institution/120127'
s = pagesource(url, d)

In [22]:
s1 = pagesource(url1, d)
s1
url2 = 'https://freida.ama-assn.org/institution/128075'
s2 = pagesource(url2, d)

In [24]:
mc = list(s2.findAll('small', class_="bold"))
add = list(s2.findAll('div', class_="details__address ng-star-inserted"))
for i in range(0,len(mc)):
  if mc[i] != []:
    print(mc[i].text)
add.text

Med Coll of Georgia at Augusta Univ


AttributeError: ignored

In [ ]:
s